### Day 21: Keypad Conundrum

Link: https://adventofcode.com/2024/day/21#part2

To solve part two, we need another approach to generate the shortest directional keypad instructions. Since the keypad only has five keys, we can hard-code the shortest paths for each combination of origin and destination. We can do this so each path component is another key combination, creating a cyclic lookup. We can then count how many times each combination shows up in an instruction and add that count to the new combinations it generates. This allows us to skip counting instructions one by one, which would take a very long time and a lot of memory given the expected number is on the order of 10¹⁴.

In [1]:
# Please ensure there is an `input.txt` file in this folder containing your input.
with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
from collections import deque


codes: list[str] = []


for line in lines:
    codes.append(line.strip())


numpad = [
    ["7", "8", "9"],
    ["4", "5", "6"],
    ["1", "2", "3"],
    ["*", "0", "A"],
]
arrow_pad = [
    ["*", "^", "A"],
    ["<", "v", ">"],
]
directions = {
    "^": (-1, 0),  # Up
    ">": (0, 1),  # Right
    "v": (1, 0),  # Down
    "<": (0, -1),  # Left
}
arrow_pad_instruction_lookup = {
    "AA": [["AA"]],
    "A^": [["A<", "<A"]],
    "A>": [["Av", "vA"]],
    "Av": [["Av", "v<", "<A"], ["A<", "<v", "vA"]],
    "A<": [["Av", "v<", "<<", "<A"]],
    "^^": [["AA"]],
    "^A": [["A>", ">A"]],
    "^v": [["Av", "vA"]],
    "^>": [["A>", ">v", "vA"], ["Av", "v>", ">A"]],
    "^<": [["Av", "v<", "<A"]],
    ">>": [["AA"]],
    ">A": [["A^", "^A"]],
    ">v": [["A<", "<A"]],
    ">^": [["A^", "^<", "<A"], ["A<", "<^", "^A"]],
    "><": [["A<", "<<", "<A"]],
    "vv": [["AA"]],
    "v^": [["A^", "^A"]],
    "v>": [["A>", ">A"]],
    "v<": [["A<", "<A"]],
    "vA": [["A^", "^>", ">A"], ["A>", ">^", "^A"]],
    "<<": [["AA"]],
    "<v": [["A>", ">A"]],
    "<>": [["A>", ">>", ">A"]],
    "<^": [["A>", ">^", "^A"]],
    "<A": [["A>", ">>", ">^", "^A"]],
}


def is_within_grid(row: int, column: int, grid: list[list[str]]) -> bool:
    return 0 <= row < len(grid) and 0 <= column < len(grid[0])


def generate_keypad_instructions(
    keypad: list[list[str]],
    start_row: int,
    start_column: int,
    target_instruction: str,
    generate_all: bool = False,
) -> list[str]:
    instructions = [""]
    current_row, current_column = start_row, start_column

    for digit in target_instruction:
        to_visit = deque([(current_row, current_column, "", set())])
        new_instructions: list[str] = []

        while to_visit:
            row, column, instruction, visited = to_visit.popleft()

            if (row, column) in visited:
                continue

            if new_instructions and len(instruction) >= len(new_instructions[0]):
                break

            if keypad[row][column] == digit:
                new_instructions.append(instruction + "A")
                current_row, current_column = row, column

                if generate_all:
                    continue

                break

            for direction, (add_row, add_column) in directions.items():
                new_row, new_column = row + add_row, column + add_column

                if (
                    is_within_grid(new_row, new_column, keypad)
                    and keypad[new_row][new_column] != "*"
                ):
                    to_visit.append(
                        (
                            new_row,
                            new_column,
                            instruction + direction,
                            visited | {(row, column)},
                        )
                    )

        instructions = [
            instruction + new_instruction
            for instruction in instructions
            for new_instruction in new_instructions
        ]

    return instructions


def generate_numpad_instructions(code: str, generate_all: bool = False) -> list[str]:
    return generate_keypad_instructions(numpad, 3, 2, code, generate_all)


def generate_arrow_pad_instructions(instructions: dict[str, int]) -> list[dict[str, int]]:
    result: list[dict[str, int]] = [{"length": 0}]

    for instruction, count in instructions.items():
        if instruction == "length":
            continue

        new_result: list[dict[str, int]] = []

        for new_instructions in arrow_pad_instruction_lookup[instruction]:
            local_result = [_.copy() for _ in result]

            for new_instruction in new_instructions:
                for result_instruction in local_result:
                    result_instruction.setdefault(new_instruction, 0)
                    result_instruction[new_instruction] += count
                    result_instruction["length"] += count

            new_result.extend(local_result)

        result = new_result

    return result


complexity_sum = 0


for code in codes:
    instructions = generate_numpad_instructions(code, True)
    instructions_parts: list[dict[str, int]] = []

    for instruction in instructions:
        instruction_parts: dict[str, int] = {}
        previous_key = "A"

        for current_key in instruction:
            key = f"{previous_key}{current_key}"
            instruction_parts.setdefault(key, 0)
            instruction_parts[key] += 1
            previous_key = current_key

        instruction_parts["length"] = sum(instruction_parts.values())
        instructions_parts.append(instruction_parts)

    for _ in range(25):
        new_instructions: list[dict[str, int]] = []

        for instruction_parts in instructions_parts:
            keypad_instructions = generate_arrow_pad_instructions(instruction_parts)

            if not new_instructions or keypad_instructions[0]["length"] < new_instructions[0]["length"]:
                new_instructions = keypad_instructions
            elif keypad_instructions[0]["length"] == new_instructions[0]["length"]:
                new_instructions.extend(keypad_instructions)

        instructions_parts = new_instructions

    shortest_instruction_length = instructions_parts[0]["length"]
    complexity = shortest_instruction_length * int(code[:-1])
    complexity_sum += complexity


print(complexity_sum)